In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (
    Input, LSTM, RepeatVector, TimeDistributed, Dense,
    Concatenate, Lambda, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers


# Compute correlation matrix on the training set
corr_mat = train_scaled.corr()

# 3a. Elbow Method: compute SSE for cluster counts 1–9
sse = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=2000, random_state=42)
    kmeans.fit(corr_mat)
    sse.append(kmeans.inertia_)

# Plot the elbow curve
frame_elbow = pd.DataFrame({'Cluster': range(1, 10), 'SSE': sse})
plt.figure(figsize=(10, 5))
plt.plot(frame_elbow['Cluster'], frame_elbow['SSE'], marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('SSE (Inertia)')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()

# Silhouette Method: evaluate silhouette score for k = 2–6
silhouette_scores = {}
for k in range(2, 7):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(corr_mat)
    score = silhouette_score(corr_mat, labels)
    silhouette_scores[k] = score
    print(f"Silhouette Score for {k} clusters: {score:.4f}")

# Choose the k with the highest silhouette score
optimal_clusters = max(silhouette_scores, key=silhouette_scores.get)
print(f"\nOptimal number of clusters based on silhouette score: {optimal_clusters}")

# Cluster Features on the Correlation Matrix
# -------------------------------
kmeans_final = KMeans(n_clusters=optimal_clusters, random_state=42)
cluster_labels = kmeans_final.fit_predict(corr_mat)


# Utility Functions
# ------------------------------------------
def print_clusters(df, cluster_labels):
    """
    Print feature names belonging to each cluster.
    """
    clusters = {}
    for idx, lbl in enumerate(cluster_labels):
        clusters.setdefault(lbl, []).append(df.columns[idx])
    for lbl, feats in clusters.items():
        print(f"Cluster {lbl}: {', '.join(feats)}")


def create_clustered_dataframes(df, cluster_labels):
    """
    Given a DataFrame `df` and an array of cluster labels (one label per column),
    return a dict mapping each label → DataFrame of the columns in that cluster.
    """
    clustered = {}
    for lbl in np.unique(cluster_labels):
        cols = [col for col, lab in zip(df.columns, cluster_labels) if lab == lbl]
        clustered[lbl] = df[cols]
    return clustered


def create_sequences(clustered_dfs, seq_length):
    """
    For each clustered DataFrame in `clustered_dfs`, build a 3D array of rolling
    sequences of length `seq_length`. Returns {cluster_label: np.array(shape=(n_samples, seq_length, n_features))}.
    """
    sequences = {}
    for lbl, df in clustered_dfs.items():
        arr = df.values  # shape: (time_steps, n_features_cluster)
        xs = []
        for i in range(len(arr) - seq_length):
            xs.append(arr[i : i + seq_length])
        sequences[lbl] = np.stack(xs)  # → shape (n_samples, seq_length, n_features_cluster)
    return sequences


def sequences_to_original(X_seq, seq_length):
    """
    Reconstruct a time series from the VAE output sequences.
    X_seq: array of shape (n_samples, seq_length, n_features)
    Returns an array of shape (n_samples + seq_length, n_features) where each
    time step i is taken from X_seq[i, 0, :], and the final `seq_length` steps
    come from X_seq[-1, :, :].
    """
    n_samples, L, n_feats = X_seq.shape
    recon = np.zeros((n_samples + seq_length, n_feats))
    for i in range(n_samples):
        recon[i] = X_seq[i, 0]
    recon[n_samples:] = X_seq[-1, :, :]
    return recon


def sampling(args):
    """
    Sampling layer for VAE: z = z_mean + exp(0.5 * z_log_var) * epsilon
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


# VAE Model Definition
# ------------------------------------------
def vae_model(input_seq_list, kl_weight=0.75, latent_dim=8, dropout_rate=0.2):
    """
    Build a VAE that takes a list of input sequences (one per cluster).
    Each branch encodes its cluster-specific sequence into a latent vector,
    and then we concatenate all latent vectors and decode back to the full feature set.

    input_seq_list: list of np.arrays, each of shape (n_samples, seq_length, n_feats_cluster)
    """
    inputs = []
    latents = []
    kl_losses = []

    # Determine total number of features across all clusters
    total_features = sum([X.shape[2] for X in input_seq_list])
    seq_len = input_seq_list[0].shape[1]

    # Build one encoder branch per cluster
    for X in input_seq_list:
        inp = Input(shape=(seq_len, X.shape[2]))
        h = LSTM(64, activation='relu', return_sequences=True)(inp)
        h = Dropout(dropout_rate)(h)
        h = LSTM(32, activation='relu', return_sequences=True)(h)
        h = Dropout(dropout_rate)(h)
        h = LSTM(16, activation='relu', return_sequences=True)(h)
        h = LSTM(8, activation='relu', return_sequences=False)(h)

        z_mean = Dense(latent_dim, kernel_regularizer=regularizers.l2(1e-2))(h)
        z_log_var = Dense(latent_dim, kernel_regularizer=regularizers.l2(1e-2))(h)
        z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

        inputs.append(inp)
        latents.append(z)

        # KL divergence for this branch
        kl = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        kl_losses.append(kl)

    # Concatenate latent vectors from all clusters
    merged = Concatenate(axis=-1)(latents)
    
    dec = RepeatVector(seq_len)(merged)
    dec = LSTM(8, activation='relu', return_sequences=True)(dec)
    dec = LSTM(16, activation='relu', return_sequences=True)(dec)
    dec = LSTM(32, activation='relu', return_sequences=True)(dec)
    dec = LSTM(64, activation='relu', return_sequences=True)(dec)

    output = TimeDistributed(Dense(total_features))(dec)

    vae = Model(inputs=inputs, outputs=output)

    # Add KL loss (sum over all branches)
    total_kl = K.mean(tf.add_n(kl_losses))
    vae.add_loss(kl_weight * total_kl)

    vae.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')
    return vae



# Main Pipeline (Training & Testing)
# ------------------------------------------

# Convert column names to strings (required for concatenation later)
train.columns = train.columns.astype(str)
test.columns  = test.columns.astype(str)

# Cluster the features based on correlation
n_clusters = optimal_clusters
corr_mat = train.corr()
clustering = KMeans(n_clusters=n_clusters, random_state=0).fit(corr_mat)
cluster_labels = clustering.labels_

print("Feature clusters (training data):")
print_clusters(train, cluster_labels)

# Split `train` and `test` into cluster‐specific DataFrames
clustered_train_dfs = create_clustered_dataframes(train, cluster_labels)
clustered_test_dfs  = create_clustered_dataframes(test, cluster_labels)

# Sequence length
T = 14
np.random.seed(40)

# Build rolling sequences for each cluster
train_sequences = create_sequences(clustered_train_dfs, T)
test_sequences  = create_sequences(clustered_test_dfs, T)

# Prepare model inputs as a list of arrays, one per cluster
train_input_list = [train_sequences[lbl] for lbl in sorted(train_sequences.keys())]
test_input_list  = [test_sequences[lbl]  for lbl in sorted(test_sequences.keys())]

vae = vae_model(train_input_list, kl_weight=0.75, latent_dim=8, dropout_rate=0.2)

# We need to construct the “true” output by concatenating all cluster inputs along the feature axis:
true_train = np.concatenate(train_input_list, axis=-1)  # shape: (n_samples, T, total_features)

early_stop = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-3,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

history = vae.fit(
    train_input_list,
    true_train,         
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop],
    shuffle=False
)

# 
train_pred_seq = vae.predict(train_input_list)
train_pred_recon = sequences_to_original(train_pred_seq, T)

# Reconstruct actual training data in the same concatenated order
actual_train = pd.concat(
    [clustered_train_dfs[lbl] for lbl in sorted(clustered_train_dfs.keys())],
    axis=1
).values

n_train_samples = train_pred_seq.shape[0]
train_mse = mean_squared_error(
    actual_train[:n_train_samples], train_pred_recon[:n_train_samples]
)
print(f"Train Mean Squared Error: {train_mse:.6f}")

# 3h. Evaluate on Test Set
test_pred_seq = vae.predict(test_input_list, batch_size=256)
test_pred_recon = sequences_to_original(test_pred_seq, T)

actual_test = pd.concat(
    [clustered_test_dfs[lbl] for lbl in sorted(clustered_test_dfs.keys())],
    axis=1
).values

n_test_samples = test_pred_seq.shape[0]
test_mse = mean_squared_error(
    actual_test[:n_test_samples], test_pred_recon[:n_test_samples]
)
print(f"Test Mean Squared Error: {test_mse:.6f}")
vae.save('/home/smlt_model')
